In [2]:
import pandas as pd
import matplotlib.pyplot as pyplot

/tmp/ipykernel_851/2884498329.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Leemos el dataset que serán necesarios, en este caso el d emovies y el rating, al menos por ahora
df = pd.read_csv("../data/movies.csv")
dff = pd.read_csv("../data/ratings.csv")
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# Para ver el contenido d emanera rápida, simplemente hacemos el head
dff.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


- Revisamos como vienen los datos para saber si es necesario alguna transformación.

In [5]:
df.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [6]:
dff.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

- Considero que el userId debería ser tipo object ya que si se usa en alguna operación puede modificarse, aunque por ahora creo que no es necesario.

## Preguntas
- Cuáles fueron los géneros mejor rankeados en promedio en el año 2005?
- Cuál es el top 5 de películas mejor rankeadas en promedio?
- cuál es el genero con más reviews?
- Cuál es el género por año con mejor rating entre el año 2000 y 2010?
- Cuál es el usuario con más ccríticas y cuál es su género favorito?

- 1. ¿Cuáles fueron los géneros mejor rankeados en promedio en el año 2005?

In [7]:
# Convertir timestamp a formato de fecha
dff['timestamp'] = pd.to_datetime(dff['timestamp'], unit='s')

In [8]:
# Como quiero filtrar por año lo agrego
dff['year'] = dff['timestamp'].dt.year

In [9]:
dff.head(1)

,userId,movieId,rating,timestamp,year
0,1,1,4.0,2000-07-30 18:45:03,2000


In [10]:
# Filtramos las filas para el año 2005
# pppp = dff.sort_values(by='year', ascending=False)
dff_del_2005 = dff[dff['year'] == 2005]
AVG_por_genero = dff_del_2005.groupby(df['genres'])['rating'].mean()

In [11]:
# Ordenamos las salidas para ver cuales son los mas altos
p1 = AVG_por_genero.sort_values(ascending=False)
p1.head(10)

genres
Children|Comedy|Romance                  5.000000
Action|Adventure|Animation|Horror        5.000000
Adventure|Drama|Romance                  5.000000
Comedy|Sci-Fi|Thriller                   5.000000
Action|Adventure|Horror|Sci-Fi           5.000000
Action|Drama|Thriller                    4.666667
Action|Drama|Mystery|Romance|Thriller    4.500000
Action|Drama|Romance|War                 4.500000
Adventure|Drama|War                      4.500000
Comedy|Drama|Horror                      4.500000
Name: rating, dtype: float64

- 2. ¿Cuál es el top 5 de películas mejor rankeadas en promedio?

- Para no tener los dos dataframes separados los unire para no tener confusión por mi.

In [12]:
data = pd.merge(df, dff, on='movieId')
data.head(3)

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,2000-07-30 18:45:03,2000
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1996-11-08 06:36:02,1996
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,2005-01-25 06:52:26,2005


In [13]:
AVG_por_title = data.groupby(["title", "year"])['rating'].mean()
AVG_por_title.head(10)

title                                    year
'71 (2014)                               2017    4.0
'Hellboy': The Seeds of Creation (2004)  2012    4.0
'Round Midnight (1986)                   2012    3.5
'Salem's Lot (2004)                      2012    5.0
'Til There Was You (1997)                2001    3.0
                                         2012    5.0
'Tis the Season for Love (2015)          2016    1.5
'burbs, The (1989)                       2000    2.6
                                         2001    5.0
                                         2003    2.0
Name: rating, dtype: float64

In [14]:
# ordenamos de forma descendente
p2 = AVG_por_title.sort_values(ascending=False)
p2.head(5)

title                                                             year
Hello, Dolly! (1969)                                              2015    5.0
Monty Python's And Now for Something Completely Different (1971)  2015    5.0
Farewell My Concubine (Ba wang bie ji) (1993)                     1999    5.0
All About Eve (1950)                                              1998    5.0
                                                                  1997    5.0
Name: rating, dtype: float64

- 3. ¿Cuál es el genero con más reviews?

In [35]:
p3 = data.groupby('genres')['rating'].count()
p3 = p3.sort_values(ascending=False)
p3

genres
Comedy                                    7196
Drama                                     6291
Comedy|Romance                            3967
Comedy|Drama|Romance                      3000
Comedy|Drama                              2851
                                          ... 
Crime|Drama|Film-Noir|Romance                1
Crime|Drama|Film-Noir|Romance|Thriller       1
Adventure|Fantasy|Musical|Romance            1
Crime|Drama|Horror|Mystery                   1
Comedy|Romance|Western                       1
Name: rating, Length: 951, dtype: int64

In [37]:
p3_1 = p3.idxmax()
p3_1

'Comedy'

- 4. ¿Cuál es el género por año con mejor rating entre el año 2000 y 2010?

In [15]:
# Filtra los datos para el período de 2000 a 2010
entre_years = data[(data['year'] >= 2000) & (data['year'] <= 2010)]

In [16]:
#data.count()

In [17]:
#entre_year.count()

In [18]:
peli_per_year = entre_years.groupby(['year', 'genres'])['rating'].mean().groupby('year').idxmax()
peli_per_year.head(11)
p4 = peli_per_year
p4

year
2000               (2000, Animation|Comedy|Drama|Fantasy)
2001                         (2001, Action|Drama|Western)
2002     (2002, Action|Adventure|Animation|Horror|Sci-Fi)
2003     (2003, Action|Adventure|Animation|Drama|Fantasy)
2004    (2004, Adventure|Children|Comedy|Fantasy|Musical)
2005                   (2005, Adventure|Animation|Comedy)
2006                   (2006, Action|Romance|War|Western)
2007    (2007, Adventure|Animation|Children|Comedy|Sci...
2008    (2008, Adventure|Animation|Children|Comedy|Sci...
2009    (2009, Action|Adventure|Mystery|Romance|Thriller)
2010                        (2010, Action|Comedy|Musical)
Name: rating, dtype: object

- 5. ¿Cuál es el usuario con más críticas y cuál es su género favorito?

In [33]:
# rating como critica de calificación 

# agrupamos por usuario y contamos la cantidad de cada una de las criticas, donde sacamos el id que tenga el valor máximo de estos.
numero_criticas = data.groupby('userId')['rating'].count().idxmax()

# filtramos para dejar a nuestro usiario con mayor criticas 
usuario_con_mayor_critica = data[data['userId'] == numero_criticas]

# Así agrupamos los datos de nuestro usuario por los generos que ha calificado, sacandole la media respecto al rating que les ha dado el usuario
# devolviendonos al mayor promedio de los generos que más le gustan o ha visto
gfavorito_user_mayor_critica = usuario_con_mayor_critica.groupby('genres')['rating'].mean().idxmax()

print(f'El usuario con mayor cantidad de críticas es el usuario: {numero_criticas}')
print(f'El género favorito del usuario es: {gfavorito_user_mayor_critica}')

El usuario con mayor cantidad de críticas es el usuario: 414
El género favorito del usuario es: Action|Adventure|Animation|Children|Comedy|Fantasy


- Cantidad de peliculas por género

In [45]:
peli_per_genre = data['genres'].value_counts()
peli_per_genre.head(20)

genres
Comedy                              7196
Drama                               6291
Comedy|Romance                      3967
Comedy|Drama|Romance                3000
Comedy|Drama                        2851
Drama|Romance                       2838
Action|Adventure|Sci-Fi             2361
Crime|Drama                         2315
Action|Crime|Thriller               1554
Action|Adventure|Thriller           1455
Action|Adventure|Sci-Fi|Thriller    1446
Drama|Thriller                      1365
Action|Sci-Fi|Thriller              1195
Comedy|Crime                        1171
Crime|Drama|Thriller                1119
Drama|War                           1044
Action|Drama|War                    1034
Action|Crime|Drama|Thriller         1007
Documentary                          891
Drama|Mystery|Thriller               712
Name: count, dtype: int64

- sí sepearamos los generos ¿cuál sería?

In [48]:
data2['generos'] = data['genres'].str.split('|')

In [50]:
data2.head(11)

0     [Adventure, Animation, Children, Comedy, Fantasy]
1     [Adventure, Animation, Children, Comedy, Fantasy]
2     [Adventure, Animation, Children, Comedy, Fantasy]
3     [Adventure, Animation, Children, Comedy, Fantasy]
4     [Adventure, Animation, Children, Comedy, Fantasy]
5     [Adventure, Animation, Children, Comedy, Fantasy]
6     [Adventure, Animation, Children, Comedy, Fantasy]
7     [Adventure, Animation, Children, Comedy, Fantasy]
8     [Adventure, Animation, Children, Comedy, Fantasy]
9     [Adventure, Animation, Children, Comedy, Fantasy]
10    [Adventure, Animation, Children, Comedy, Fantasy]
Name: genres, dtype: object

In [57]:
data2['generos'].value_counts()

genres
[Comedy]                                7196
[Drama]                                 6291
[Comedy, Romance]                       3967
[Comedy, Drama, Romance]                3000
[Comedy, Drama]                         2851
                                        ... 
[Crime, Drama, Film-Noir, Romance]         1
[Crime, Romance]                           1
[Adventure, Documentary, Western]          1
[Action, Fantasy, Western]                 1
[Action, Animation, Comedy, Fantasy]       1
Name: count, Length: 951, dtype: int64

- Por lo visto no cambia de esta manera, quiza realizando un par de cosas más podríamos descubrir algo.

In [59]:
data.head()

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,2000-07-30 18:45:03,2000
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1996-11-08 06:36:02,1996
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,2005-01-25 06:52:26,2005
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,2017-11-13 12:59:30,2017
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,2011-05-18 05:28:03,2011


In [61]:
data_2 = data.sort_values(by="year",ascending=False)
data_2.head(12)

,movieId,title,genres,userId,rating,timestamp,year
50418,2991,Live and Let Die (1973),Action|Adventure|Thriller,382,3.5,2018-01-05 17:34:39,2018
67981,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,Adventure|Animation|Fantasy,98,4.0,2018-07-24 18:49:21,2018
68006,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,Adventure|Animation|Fantasy,306,3.5,2018-02-11 20:18:03,2018
68027,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,Adventure|Animation|Fantasy,459,5.0,2018-03-05 07:06:55,2018
68043,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,Adventure|Animation|Fantasy,567,2.0,2018-05-02 19:13:48,2018
68045,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,Adventure|Animation|Fantasy,596,4.5,2018-08-31 09:43:40,2018
68048,5618,Spirited Away (Sen to Chihiro no kamikakushi) ...,Adventure|Animation|Fantasy,601,5.0,2018-03-18 18:26:40,2018
68061,5620,Sweet Home Alabama (2002),Comedy|Romance,111,3.0,2018-01-16 22:21:04,2018
68072,5620,Sweet Home Alabama (2002),Comedy|Romance,382,3.5,2018-01-05 17:21:30,2018
68083,5621,"Tuxedo, The (2002)",Action|Comedy,89,5.0,2018-03-07 07:41:14,2018


- ¿Cuáles serían las peliículas del año más reciente que menos calificación tuvieron?
Parece ser que el último año es el 2018.

In [64]:
last_year = data[data['year'] >= 2018]
last_year

,movieId,title,genres,userId,rating,timestamp,year
30,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,89,3.0,2018-03-07 07:38:34,2018
35,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,98,4.5,2018-07-24 18:44:09,2018
52,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,153,2.0,2018-05-05 19:30:42,2018
84,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,233,3.0,2018-04-26 22:20:49,2018
137,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,382,4.5,2018-01-05 14:30:28,2018
...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,2018-09-16 14:44:42,2018
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,2018-09-16 14:52:25,2018
100833,193585,Flint (2017),Drama,184,3.5,2018-09-16 14:56:45,2018
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,2018-09-16 15:00:21,2018


In [75]:
pelis_menor_rank = last_year.groupby(["title", "genres", "timestamp"])['rating'].mean()

pi = pelis_menor_rank.sort_values(ascending=True)
pi.head(20)

title                                                    genres                               timestamp          
Boudu Saved From Drowning (Boudu sauvé des eaux) (1932)  Comedy                               2018-05-02 19:40:50    0.5
Silence of the Lambs, The (1991)                         Crime|Horror|Thriller                2018-01-16 22:11:47    0.5
Jack and Jill (2011)                                     Comedy                               2018-05-02 19:38:51    0.5
Tooth Fairy 2 (2012)                                     Children|Comedy                      2018-05-02 19:36:25    0.5
Jennifer's Body (2009)                                   Comedy|Horror|Sci-Fi|Thriller        2018-01-17 01:31:38    0.5
Jesus Christ Vampire Hunter (2001)                       Action|Comedy|Horror|Musical         2018-02-21 01:13:16    0.5
Tomb Raider (2018)                                       Action|Adventure|Fantasy             2018-05-05 20:44:11    0.5
To Be or Not to Be (1942)              

In [66]:
data.columns.values

array(['movieId', 'title', 'genres', 'userId', 'rating', 'timestamp',
       'year'], dtype=object)

In [ ]:
# cuales son las peliculas con mas antiguedad